### Table of contents
1. [Supp Table 1. Table of cell identities and clinical metadata](#1)
2. [Supp Table 2. All gene sets used in study](#2)
3. [Supp Table 3. Significant mixed effects IFN-inducibility model results for all cell types in full dataset (tx and no-tx)](#3)
4. [Supp Table 4. DEG between CD8 subsets](#4)
5. [Supp Table 5. DEG between non-cycling TAM subsets](#5)
6. [Supp Table 6. Significant mixed effects IFN-inducibility model results for all cell types in untreated subset of dataset](#6)

In [1]:
library(harmony)
library(stringr)
library(Seurat)
library(useful)
library(dplyr)
library(ggpubr)
library(glue)
library(RColorBrewer)
library(ggpubr)
library(ggrepel)
library(lmerTest)
library(Scillus)
library(cowplot)
library(hrbrthemes)

set.seed(1234)

options(repr.plot.width=12, repr.plot.height=9)

Loading required package: Rcpp

Attaching SeuratObject

Attaching sp

Loading required package: ggplot2


Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Loading required package: lme4

Loading required package: Matrix


Attaching package: 'lmerTest'


The following object is masked from 'package:lme4':

    lmer


The following object is masked from 'package:stats':

    step



Attaching package: 'cowplot'


The following object is masked from 'package:ggpubr':

    get_legend


NOTE: Either Arial Narrow or Roboto Condensed fonts are required to use these themes.

      Please use hrbrthemes::import_roboto_condensed() to install Roboto Condensed and

      if Arial Narrow is not on your system, please see https://bit.ly/arialnarrow



### Supp Table 1. Table of cell identities and clinical metadata <a name="1"></a>

In [8]:
seur_full = readRDS(file = "../../Data_Objects/seur_full_BBK_Manuscript.RDS")

In [32]:
features_select = c("nCount_RNA","nFeature_RNA","Sample","Patient","Cohort","Chemistry","Biopsy","Stage_Corrected","Stage_Corrected_EarlyLate","IO_Treated","IO","TKI","InferCNV_Call_Manuscript","BBK_Broad_Celltype","BBK_Broad_Celltype_DE","BBK_Final_Celltype")
meta = seur_full@meta.data[,features_select]
dim(meta)

[1] 288768     16

In [33]:
head(meta)

,nCount_RNA,nFeature_RNA,Sample,Patient,Cohort,Chemistry,Biopsy,Stage_Corrected,Stage_Corrected_EarlyLate,IO_Treated,IO,TKI,InferCNV_Call_Manuscript,BBK_Broad_Celltype,BBK_Broad_Celltype_DE,BBK_Final_Celltype
,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<fct>,<fct>,<fct>
CCACGGAGTCTACCTC-1.5pool1_2,21107,5193,5pool1_2,S1,Braun,5prime,Kidney,I,Early,IO Naive,No,No,Malignant,Omitted,Omitted,Omitted
GTTCTCGAGAGGTAGA-1.5pool1_2,17633,4215,5pool1_2,S1,Braun,5prime,Kidney,I,Early,IO Naive,No,No,Malignant,Tumor,Tumor,Tumor_TNFSF10
AGCTCTCGTGACTCAT-1.5pool1_2,15906,4217,5pool1_2,S1,Braun,5prime,Kidney,I,Early,IO Naive,No,No,Malignant,Omitted,Omitted,Omitted
GGAAAGCCATCATCCC-1.5pool1_2,15104,4042,5pool1_2,S1,Braun,5prime,Kidney,I,Early,IO Naive,No,No,Malignant,Omitted,Omitted,Omitted
CTAGAGTCATCAGTCA-1.5pool1_2,14519,4068,5pool1_2,S1,Braun,5prime,Kidney,I,Early,IO Naive,No,No,Malignant,Tumor,Tumor,Tumor_CXCL14
CGGCTAGAGGTAGCCA-1.5pool1_2,14443,3521,5pool1_2,S1,Braun,5prime,Kidney,I,Early,IO Naive,No,No,Malignant,Tumor,Tumor,Tumor_CXCL14


### Read in ISG.RS and IFNG.GS signature scores

In [29]:
res = data.frame(readRDS("../../Figures/Figure1/seur_full_isg.rs_ifng.gs_scores_hacksig_zscore.RDS"))
row.names(res) = res$sample_id
corner(res)

,sample_id,Minn_Benci_ISG.RS,Minn_Benci_IFNG.GS
,<chr>,<dbl>,<dbl>
GTTCTCGAGAGGTAGA-1.5pool1_2,GTTCTCGAGAGGTAGA-1.5pool1_2,0.03554675,1.52951884
CTAGAGTCATCAGTCA-1.5pool1_2,CTAGAGTCATCAGTCA-1.5pool1_2,0.63826751,0.66786794
CGGCTAGAGGTAGCCA-1.5pool1_2,CGGCTAGAGGTAGCCA-1.5pool1_2,-0.60394903,0.02734308
CTGCTGTAGTGTTTGC-1.5pool1_2,CTGCTGTAGTGTTTGC-1.5pool1_2,0.18163262,1.98420733
CCATGTCCATCGATGT-1.5pool1_2,CCATGTCCATCGATGT-1.5pool1_2,0.79286452,0.75634495


In [34]:
meta$barcode_join = row.names(meta)

In [35]:
res$barcode_join = row.names(res)

In [39]:
meta = left_join(meta, res[,c("Minn_Benci_ISG.RS","Minn_Benci_IFNG.GS","barcode_join")],by = "barcode_join")

In [42]:
row.names(meta) = meta$barcode_join

In [43]:
head(meta)

,nCount_RNA,nFeature_RNA,Sample,Patient,Cohort,Chemistry,Biopsy,Stage_Corrected,Stage_Corrected_EarlyLate,IO_Treated,IO,TKI,InferCNV_Call_Manuscript,BBK_Broad_Celltype,BBK_Broad_Celltype_DE,BBK_Final_Celltype,barcode_join,Minn_Benci_ISG.RS,Minn_Benci_IFNG.GS
,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<fct>,<fct>,<fct>,<chr>,<dbl>,<dbl>
CCACGGAGTCTACCTC-1.5pool1_2,21107,5193,5pool1_2,S1,Braun,5prime,Kidney,I,Early,IO Naive,No,No,Malignant,Omitted,Omitted,Omitted,CCACGGAGTCTACCTC-1.5pool1_2,NA,NA
GTTCTCGAGAGGTAGA-1.5pool1_2,17633,4215,5pool1_2,S1,Braun,5prime,Kidney,I,Early,IO Naive,No,No,Malignant,Tumor,Tumor,Tumor_TNFSF10,GTTCTCGAGAGGTAGA-1.5pool1_2,0.03554675,1.52951884
AGCTCTCGTGACTCAT-1.5pool1_2,15906,4217,5pool1_2,S1,Braun,5prime,Kidney,I,Early,IO Naive,No,No,Malignant,Omitted,Omitted,Omitted,AGCTCTCGTGACTCAT-1.5pool1_2,NA,NA
GGAAAGCCATCATCCC-1.5pool1_2,15104,4042,5pool1_2,S1,Braun,5prime,Kidney,I,Early,IO Naive,No,No,Malignant,Omitted,Omitted,Omitted,GGAAAGCCATCATCCC-1.5pool1_2,NA,NA
CTAGAGTCATCAGTCA-1.5pool1_2,14519,4068,5pool1_2,S1,Braun,5prime,Kidney,I,Early,IO Naive,No,No,Malignant,Tumor,Tumor,Tumor_CXCL14,CTAGAGTCATCAGTCA-1.5pool1_2,0.63826751,0.66786794
CGGCTAGAGGTAGCCA-1.5pool1_2,14443,3521,5pool1_2,S1,Braun,5prime,Kidney,I,Early,IO Naive,No,No,Malignant,Tumor,Tumor,Tumor_CXCL14,CGGCTAGAGGTAGCCA-1.5pool1_2,-0.60394903,0.02734308


In [46]:
write.table(meta, file = "../../Tables/Supp_Table1_CellMetadata.txt", sep = '\t',quote = F, col.names = NA)

### Supp Table 2. All gene sets used in study <a name="2"></a>

In [47]:
putative_isgs = readRDS("../../Signatures/manuscript_goi_hallmark_benciminn_curated.RDS")
length(putative_isgs)

[1] 259

In [49]:
other_sigs = readRDS(file = "../../Signatures/Final_Manuscript_Signatures_CTS_ISGs.RDS")
length(other_sigs)

[1] 240

In [50]:
names(other_sigs)

[1] "manuscript_IFN1_myeloid"                      
  [2] "manuscript_IFN1_t_nk"                         
  [3] "manuscript_IFN1_tumor"                        
  [4] "manuscript_IFNG_myeloid"                      
  [5] "manuscript_IFNG_t_nk"                         
  [6] "manuscript_IFNG_tumor"                        
  [7] "manuscript_myeloid_combined"                  
  [8] "manuscript_t_nk_combined"                     
  [9] "manuscript_tumor_combined"                    
 [10] "TAM_NISG"                                     
 [11] "Monocyte_NISG"                                
 [12] "DC_NISG"                                      
 [13] "Tumor_NISG"                                   
 [14] "Perivascular_and_Fibroblast_NISG"             
 [15] "IFNg score"                                   
 [16] "Expanded immune gene signature"               
 [17] "T cell inflamed GEP"                          
 [18] "IMPRES signature"                             
 [19] "CYT score"                                    
 [20] "T exhaust signature"                          
 [21] "T accum signature"                            
 [22] "CTL signature"                                
 [23] "TIDE signature"                               
 [24] "csCD8_Tcell_signature_37043555"               
 [25] "IR7_score"                                    
 [26] "PDL1_data"                                    
 [27] "Buck14_score"                                 
 [28] "ICS5_score"                                   
 [29] "LIexpression_score"                           
 [30] "Chemokine12_score"                            
 [31] "NHI_5gene_score"                              
 [32] "PD1_data"                                     
 [33] "CD8_CD68_ratio"                               
 [34] "CD68"                                         
 [35] "CD8A"                                         
 [36] "PD1_PDL1_score"                               
 [37] "CTLA4_data"                                   
 [38] "Bcell_mg_IGJ"                                 
 [39] "Bcell_receptors_score"                        
 [40] "STAT1_score"                                  
 [41] "CSF1_response"                                
 [42] "Module3_IFN_score"                            
 [43] "Module4_TcellBcell_score"                     
 [44] "Module5_TcellBcell_score"                     
 [45] "Module11_Prolif_score"                        
 [46] "TAMsurr_score"                                
 [47] "TcClassII_score"                              
 [48] "TAMsurr_TcClassII_ratio"                      
 [49] "IL12_score_21050467"                          
 [50] "IL4_score_21050467"                           
 [51] "IL2_score_21050467"                           
 [52] "IL13_score_21050467"                          
 [53] "IFNG_score_21050467"                          
 [54] "TGFB_score_21050467"                          
 [55] "TREM1_data"                                   
 [56] "DAP12_data"                                   
 [57] "Immune_NSCLC_score"                           
 [58] "Tcell_receptors_score"                        
 [59] "CHANG_CORE_SERUM_RESPONSE_UP"                 
 [60] "CSR_Activated_15701700"                       
 [61] "KEGG_HEMATOPOIETIC_CELL_LINEAGE"              
 [62] "IgG_19272155"                                 
 [63] "Interferon_19272155"                          
 [64] "LCK_19272155"                                 
 [65] "MHC.I_19272155"                               
 [66] "MHC.II_19272155"                              
 [67] "STAT1_19272155"                               
 [68] "Troester_WoundSig_19887484"                   
 [69] "MDACC.FNA.1_20805453"                         
 [70] "IGG_Cluster_21214954"                         
 [71] "Minterferon_Cluster_21214954"                 
 [72] "ZHANG_INTERFERON_RESPONSE"                    
 [73] "Immune_cell_Cluster_21214954"                 
 [74] "MCD3_CD8_21214954"                            
 [75] 

In [55]:
supp_table2_sigs = list()

In [56]:
supp_table2_sigs$Putative_ISGs = putative_isgs

In [58]:
supp_table2_sigs = c(supp_table2_sigs, other_sigs[c("Minn_Benci_ISG.RS","Minn_Benci_IFNG.GS")])
supp_table2_sigs

$Putative_ISGs
  [1] "ADAR"     "B2M"      "BATF2"    "BST2"     "C1S"      "CASP1"   
  [7] "CASP8"    "CCRL2"    "CD47"     "CD74"     "CMPK2"    "CMTR1"   
 [13] "CNP"      "CSF1"     "CXCL10"   "CXCL11"   "DDX60"    "DHX58"   
 [19] "EIF2AK2"  "ELF1"     "EPSTI1"   "GBP2"     "GBP4"     "GMPR"    
 [25] "HELZ2"    "HERC6"    "HLA-C"    "IFI27"    "IFI30"    "IFI35"   
 [31] "IFI44"    "IFI44L"   "IFIH1"    "IFIT2"    "IFIT3"    "IFITM1"  
 [37] "IFITM2"   "IFITM3"   "IL15"     "IL4R"     "IL7"      "IRF1"    
 [43] "IRF2"     "IRF7"     "IRF9"     "ISG15"    "ISG20"    "LAMP3"   
 [49] "LAP3"     "LGALS3BP" "LPAR6"    "LY6E"     "MOV10"    "MVB12A"  
 [55] "MX1"      "NCOA7"    "NMI"      "NUB1"     "OAS1"     "OASL"    
 [61] "OGFR"     "PARP12"   "PARP14"   "PARP9"    "PLSCR1"   "PNPT1"   
 [67] "PROCR"    "PSMA3"    "PSMB8"    "PSMB9"    "PSME1"    "PSME2"   
 [73] "RIPK2"    "RNF31"    "RSAD2"    "RTP4"     "SAMD9"    "SAMD9L"  
 [79] "SELL"     "SLC25A28" "SP110"    "STAT2"    "TAP1"     "TDRD7"   
 [85] "TENT5A"   "TMEM140"  "TRAFD1"   "TRIM14"   "TRIM21"   "TRIM25"  
 [91] "TRIM26"   "TRIM5"    "TXNIP"    "UBA7"     "UBE2L6"   "USP18"   
 [97] "WARS1"    "APOL6"    "ARID5B"   "ARL4A"    "AUTS2"    "BANK1"   
[103] "BPGM"     "BTG1"     "C1R"      "CASP3"    "CASP4"    "CASP7"   
[109] "CCL2"     "CCL5"     "CCL7"     "CD274"    "CD38"     "CD40"    
[115] "CD69"     "CD86"     "CDKN1A"   "CFB"      "CFH"      "CIITA"   
[121] "CMKLR1"   "CSF2RB"   "CXCL9"    "EIF4E3"   "FAS"      "FCGR1A"  
[127] "FGL2"     "FPR1"     "GBP6"     "GCH1"     "GPR18"    "GZMA"    
[133] "HIF1A"    "HLA-A"    "HLA-B"    "HLA-DMA"  "HLA-DQA1" "HLA-DRB1"
[139] "HLA-G"    "ICAM1"    "IDO1"     "IFIT1"    "IFNAR2"   "IL10RA"  
[145] "IL15RA"   "IL18BP"   "IL2RB"    "IL6"      "IRF4"     "IRF5"    
[151] "IRF8"     "ISOC1"    "ITGB7"    "JAK2"     "KLRK1"    "LATS2"   
[157] "LCP2"     "LYSMD2"   "MARCHF1"  "METTL7B"  "MT2A"     "MTHFD2"  
[163] "MVP"      "MX2"      "MYD88"    "NAMPT"    "NCOA3"    "NFKB1"   
[169] "NFKBIA"   "NLRC5"    "NOD1"     "NUP93"    "OAS2"     "OAS3"    
[175] "P2RY14"   "PDE4B"    "PELI1"    "PFKP"     "PIM1"     "PLA2G4A" 
[181] "PML"      "PNP"      "PSMA2"    "PSMB10"   "PSMB2"    "PTGS2"   
[187] "PTPN1"    "PTPN2"    "PTPN6"    "RAPGEF6"  "RBCK1"    "RIGI"    
[193] "RIPK1"    "RNF213"   "SAMHD1"   "SECTM1"   "SELP"     "SERPING1"
[199] "SLAMF7"   "SOCS1"    "SOCS3"    "SOD2"     "SPPL2A"   "SRI"     
[205] "SSPN"     "ST3GAL5"  "ST8SIA4"  "STAT1"    "STAT3"    "STAT4"   
[211] "TAPBP"    "TNFAIP2"  "TNFAIP3"  "TNFAIP6"  "TNFSF10"  "TOR1B"   
[217] "UPP1"     "VAMP5"    "VAMP8"    "VCAM1"    "XAF1"     "XCL1"    
[223] "ZBP1"     "ZNFX1"    "HSD17B1"  "CA2"      "CCNA1"    "CXCL1"   
[229] "GALC"     "IFI6"     "MCL1"     "ROBO1"    "SLC6A15"  "THBS1"   
[235] "TIMP3"    "DDX58"    "WARS"     "MARCH1"   "PRIC285"  "FTSJD2"  
[241] "NOS2"     "PDCD1LG2" "LGALS9"   "HMGB1"    "CEACAM1"  "FGL1"    
[247] "HLA-DRA"  "NECTIN2"  "PVR"      "SIGLEC10" "TNFRSF14" "VSIG4"   
[253] "VSIR"     "HLA-E"    "LGALS3"   "CD80"     "LILRB1"   "IL10"    
[259] "CCL22"   

$Minn_Benci_ISG.RS
 [1] "IFI27"    "IRF7"     "USP18"    "BST2"     "CXCL10"   "DDX60"   
 [7] "HERC6"    "HLA-B"    "HLA-G"    "IFI35"    "IFI44"    "IFI44L"  
[13] "IFIT1"    "IFIT3"    "ISG15"    "LGALS3BP" "LY6E"     "MX1"     
[19] "MX2"      "OAS3"     "OASL"     "PLSCR1"   "STAT1"    "TRIM14"  
[25] "HSD17B1"  "OAS1"     "CA2"      "CCNA1"    "CXCL1"    "GALC"    
[31] "IFI6"     "IFITM1"   "LAMP3"    "MCL1"     "ROBO1"    "SLC6A15" 
[37] "THBS1"    "TIMP3"   

$Minn_Benci_IFNG.GS
  [1] "TNFSF10"  "IRF9"     "PSMB9"    "EPSTI1"   "PARP12"   "TRIM25"  
  [7] "LAP3"     "CASP7"    "UPP1"     "B2M"      "IRF4"     "SRI"     
 [13] "NFKBIA"   "IFIT2"    "OAS2"     "TAP1"     "EIF2AK2"  "RSAD2"   
 [19] "IRF1"     "XAF1"     "SP110"    "PSMB8"    "IFITM3"   "GBP4"    
 [25] "IRF8"     "PML"      "IFIH1"    "UBE2L6"   "ADAR"     "STAT2"   
 [31] "CXCL9"    "

In [59]:
grep("manuscript",names(other_sigs),value = T)

[1] "manuscript_IFN1_myeloid"     "manuscript_IFN1_t_nk"       
[3] "manuscript_IFN1_tumor"       "manuscript_IFNG_myeloid"    
[5] "manuscript_IFNG_t_nk"        "manuscript_IFNG_tumor"      
[7] "manuscript_myeloid_combined" "manuscript_t_nk_combined"   
[9] "manuscript_tumor_combined"

In [60]:
cts_isg_sigs = other_sigs[grep("manuscript",names(other_sigs),value = T)]
cts_isg_sigs

$manuscript_IFN1_myeloid
 [1] "IFI30"    "LPAR6"    "FCGR1A"   "MYD88"    "VSIG4"    "LILRB1"  
 [7] "IL10"     "SECTM1"   "MX2"      "CXCL10"   "CD86"     "GCH1"    
[13] "SIGLEC10"

$manuscript_IFN1_t_nk
[1] "NLRC5" "OASL"  "IL2RB" "XCL1"  "GZMA"  "STAT4" "CCL5" 

$manuscript_IFN1_tumor
[1] "C1R"     "PFKP"    "TNFAIP6"

$manuscript_IFNG_myeloid
 [1] "CD86"     "FCGR1A"   "FPR1"     "MYD88"    "SIGLEC10" "LILRB1"  
 [7] "SECTM1"   "JAK2"     "CXCL10"   "CXCL9"    "TNFAIP2"  "PTGS2"   
[13] "IL15"     "GCH1"     "MX2"      "LAMP3"    "NFKB1"   

$manuscript_IFNG_t_nk
[1] "CCL5"  "GZMA"  "IL2RB" "NLRC5" "OASL"  "STAT4" "SOCS1"

$manuscript_IFNG_tumor
[1] "C1S"  "C1R"  "CFB"  "SSPN"

$manuscript_myeloid_combined
 [1] "IFI30"    "LPAR6"    "FCGR1A"   "MYD88"    "VSIG4"    "LILRB1"  
 [7] "IL10"     "SECTM1"   "MX2"      "CXCL10"   "CD86"     "GCH1"    
[13] "SIGLEC10" "FPR1"     "JAK2"     "CXCL9"    "TNFAIP2"  "PTGS2"   
[19] "IL15"     "LAMP3"    "NFKB1"   

$manuscript_t_nk_combined
[1] "NLRC5" "OASL"  "IL2RB" "XCL1"  "GZMA"  "STAT4" "CCL5"  "SOCS1"

$manuscript_tumor_combined
[1] "C1R"     "PFKP"    "TNFAIP6" "C1S"     "CFB"     "SSPN"

In [61]:
names(cts_isg_sigs) = c("Myeloid_IFNab_signaling","T_NK_IFNab_signaling","Tumor_IFNab_signaling","Myeloid_IFNg_signaling","T_NK_IFNg_signaling","Tumor_IFNg_signaling","Myeloid_IFNab_IFNg_signaling","T_NK_IFNab_IFNg_signaling","Tumor_IFNab_IFNg_signaling")

In [62]:
cts_isg_sigs

$Myeloid_IFNab_signaling
 [1] "IFI30"    "LPAR6"    "FCGR1A"   "MYD88"    "VSIG4"    "LILRB1"  
 [7] "IL10"     "SECTM1"   "MX2"      "CXCL10"   "CD86"     "GCH1"    
[13] "SIGLEC10"

$T_NK_IFNab_signaling
[1] "NLRC5" "OASL"  "IL2RB" "XCL1"  "GZMA"  "STAT4" "CCL5" 

$Tumor_IFNab_signaling
[1] "C1R"     "PFKP"    "TNFAIP6"

$Myeloid_IFNg_signaling
 [1] "CD86"     "FCGR1A"   "FPR1"     "MYD88"    "SIGLEC10" "LILRB1"  
 [7] "SECTM1"   "JAK2"     "CXCL10"   "CXCL9"    "TNFAIP2"  "PTGS2"   
[13] "IL15"     "GCH1"     "MX2"      "LAMP3"    "NFKB1"   

$T_NK_IFNg_signaling
[1] "CCL5"  "GZMA"  "IL2RB" "NLRC5" "OASL"  "STAT4" "SOCS1"

$Tumor_IFNg_signaling
[1] "C1S"  "C1R"  "CFB"  "SSPN"

$Myeloid_IFNab_IFNg_signaling
 [1] "IFI30"    "LPAR6"    "FCGR1A"   "MYD88"    "VSIG4"    "LILRB1"  
 [7] "IL10"     "SECTM1"   "MX2"      "CXCL10"   "CD86"     "GCH1"    
[13] "SIGLEC10" "FPR1"     "JAK2"     "CXCL9"    "TNFAIP2"  "PTGS2"   
[19] "IL15"     "LAMP3"    "NFKB1"   

$T_NK_IFNab_IFNg_signaling
[1] "NLRC5" "OASL"  "IL2RB" "XCL1"  "GZMA"  "STAT4" "CCL5"  "SOCS1"

$Tumor_IFNab_IFNg_signaling
[1] "C1R"     "PFKP"    "TNFAIP6" "C1S"     "CFB"     "SSPN"

In [63]:
supp_table2_sigs = c(supp_table2_sigs, cts_isg_sigs)
supp_table2_sigs

$Putative_ISGs
  [1] "ADAR"     "B2M"      "BATF2"    "BST2"     "C1S"      "CASP1"   
  [7] "CASP8"    "CCRL2"    "CD47"     "CD74"     "CMPK2"    "CMTR1"   
 [13] "CNP"      "CSF1"     "CXCL10"   "CXCL11"   "DDX60"    "DHX58"   
 [19] "EIF2AK2"  "ELF1"     "EPSTI1"   "GBP2"     "GBP4"     "GMPR"    
 [25] "HELZ2"    "HERC6"    "HLA-C"    "IFI27"    "IFI30"    "IFI35"   
 [31] "IFI44"    "IFI44L"   "IFIH1"    "IFIT2"    "IFIT3"    "IFITM1"  
 [37] "IFITM2"   "IFITM3"   "IL15"     "IL4R"     "IL7"      "IRF1"    
 [43] "IRF2"     "IRF7"     "IRF9"     "ISG15"    "ISG20"    "LAMP3"   
 [49] "LAP3"     "LGALS3BP" "LPAR6"    "LY6E"     "MOV10"    "MVB12A"  
 [55] "MX1"      "NCOA7"    "NMI"      "NUB1"     "OAS1"     "OASL"    
 [61] "OGFR"     "PARP12"   "PARP14"   "PARP9"    "PLSCR1"   "PNPT1"   
 [67] "PROCR"    "PSMA3"    "PSMB8"    "PSMB9"    "PSME1"    "PSME2"   
 [73] "RIPK2"    "RNF31"    "RSAD2"    "RTP4"     "SAMD9"    "SAMD9L"  
 [79] "SELL"     "SLC25A28" "SP110"    "STAT2"    "TAP1"     "TDRD7"   
 [85] "TENT5A"   "TMEM140"  "TRAFD1"   "TRIM14"   "TRIM21"   "TRIM25"  
 [91] "TRIM26"   "TRIM5"    "TXNIP"    "UBA7"     "UBE2L6"   "USP18"   
 [97] "WARS1"    "APOL6"    "ARID5B"   "ARL4A"    "AUTS2"    "BANK1"   
[103] "BPGM"     "BTG1"     "C1R"      "CASP3"    "CASP4"    "CASP7"   
[109] "CCL2"     "CCL5"     "CCL7"     "CD274"    "CD38"     "CD40"    
[115] "CD69"     "CD86"     "CDKN1A"   "CFB"      "CFH"      "CIITA"   
[121] "CMKLR1"   "CSF2RB"   "CXCL9"    "EIF4E3"   "FAS"      "FCGR1A"  
[127] "FGL2"     "FPR1"     "GBP6"     "GCH1"     "GPR18"    "GZMA"    
[133] "HIF1A"    "HLA-A"    "HLA-B"    "HLA-DMA"  "HLA-DQA1" "HLA-DRB1"
[139] "HLA-G"    "ICAM1"    "IDO1"     "IFIT1"    "IFNAR2"   "IL10RA"  
[145] "IL15RA"   "IL18BP"   "IL2RB"    "IL6"      "IRF4"     "IRF5"    
[151] "IRF8"     "ISOC1"    "ITGB7"    "JAK2"     "KLRK1"    "LATS2"   
[157] "LCP2"     "LYSMD2"   "MARCHF1"  "METTL7B"  "MT2A"     "MTHFD2"  
[163] "MVP"      "MX2"      "MYD88"    "NAMPT"    "NCOA3"    "NFKB1"   
[169] "NFKBIA"   "NLRC5"    "NOD1"     "NUP93"    "OAS2"     "OAS3"    
[175] "P2RY14"   "PDE4B"    "PELI1"    "PFKP"     "PIM1"     "PLA2G4A" 
[181] "PML"      "PNP"      "PSMA2"    "PSMB10"   "PSMB2"    "PTGS2"   
[187] "PTPN1"    "PTPN2"    "PTPN6"    "RAPGEF6"  "RBCK1"    "RIGI"    
[193] "RIPK1"    "RNF213"   "SAMHD1"   "SECTM1"   "SELP"     "SERPING1"
[199] "SLAMF7"   "SOCS1"    "SOCS3"    "SOD2"     "SPPL2A"   "SRI"     
[205] "SSPN"     "ST3GAL5"  "ST8SIA4"  "STAT1"    "STAT3"    "STAT4"   
[211] "TAPBP"    "TNFAIP2"  "TNFAIP3"  "TNFAIP6"  "TNFSF10"  "TOR1B"   
[217] "UPP1"     "VAMP5"    "VAMP8"    "VCAM1"    "XAF1"     "XCL1"    
[223] "ZBP1"     "ZNFX1"    "HSD17B1"  "CA2"      "CCNA1"    "CXCL1"   
[229] "GALC"     "IFI6"     "MCL1"     "ROBO1"    "SLC6A15"  "THBS1"   
[235] "TIMP3"    "DDX58"    "WARS"     "MARCH1"   "PRIC285"  "FTSJD2"  
[241] "NOS2"     "PDCD1LG2" "LGALS9"   "HMGB1"    "CEACAM1"  "FGL1"    
[247] "HLA-DRA"  "NECTIN2"  "PVR"      "SIGLEC10" "TNFRSF14" "VSIG4"   
[253] "VSIR"     "HLA-E"    "LGALS3"   "CD80"     "LILRB1"   "IL10"    
[259] "CCL22"   

$Minn_Benci_ISG.RS
 [1] "IFI27"    "IRF7"     "USP18"    "BST2"     "CXCL10"   "DDX60"   
 [7] "HERC6"    "HLA-B"    "HLA-G"    "IFI35"    "IFI44"    "IFI44L"  
[13] "IFIT1"    "IFIT3"    "ISG15"    "LGALS3BP" "LY6E"     "MX1"     
[19] "MX2"      "OAS3"     "OASL"     "PLSCR1"   "STAT1"    "TRIM14"  
[25] "HSD17B1"  "OAS1"     "CA2"      "CCNA1"    "CXCL1"    "GALC"    
[31] "IFI6"     "IFITM1"   "LAMP3"    "MCL1"     "ROBO1"    "SLC6A15" 
[37] "THBS1"    "TIMP3"   

$Minn_Benci_IFNG.GS
  [1] "TNFSF10"  "IRF9"     "PSMB9"    "EPSTI1"   "PARP12"   "TRIM25"  
  [7] "LAP3"     "CASP7"    "UPP1"     "B2M"      "IRF4"     "SRI"     
 [13] "NFKBIA"   "IFIT2"    "OAS2"     "TAP1"     "EIF2AK2"  "RSAD2"   
 [19] "IRF1"     "XAF1"     "SP110"    "PSMB8"    "IFITM3"   "GBP4"    
 [25] "IRF8"     "PML"      "IFIH1"    "UBE2L6"   "ADAR"     "STAT2"   
 [31] "CXCL9"    "

In [64]:
additional_sigs = c("Krishna_CD8A_Tissue_resident","tLHP_signature","MCDERMOTT_MYELOID_INFLAMMATION","Krishna_TAM_ISGInt","Krishna_TAM_ISGhi","Krishna_TAM_HLAint","Krishna_TAM_HLAhi","TAM_NISG","Monocyte_NISG","DC_NISG","TAMsurr_score")

In [65]:
supp_table2_sigs = c(supp_table2_sigs, other_sigs[additional_sigs])
supp_table2_sigs

$Putative_ISGs
  [1] "ADAR"     "B2M"      "BATF2"    "BST2"     "C1S"      "CASP1"   
  [7] "CASP8"    "CCRL2"    "CD47"     "CD74"     "CMPK2"    "CMTR1"   
 [13] "CNP"      "CSF1"     "CXCL10"   "CXCL11"   "DDX60"    "DHX58"   
 [19] "EIF2AK2"  "ELF1"     "EPSTI1"   "GBP2"     "GBP4"     "GMPR"    
 [25] "HELZ2"    "HERC6"    "HLA-C"    "IFI27"    "IFI30"    "IFI35"   
 [31] "IFI44"    "IFI44L"   "IFIH1"    "IFIT2"    "IFIT3"    "IFITM1"  
 [37] "IFITM2"   "IFITM3"   "IL15"     "IL4R"     "IL7"      "IRF1"    
 [43] "IRF2"     "IRF7"     "IRF9"     "ISG15"    "ISG20"    "LAMP3"   
 [49] "LAP3"     "LGALS3BP" "LPAR6"    "LY6E"     "MOV10"    "MVB12A"  
 [55] "MX1"      "NCOA7"    "NMI"      "NUB1"     "OAS1"     "OASL"    
 [61] "OGFR"     "PARP12"   "PARP14"   "PARP9"    "PLSCR1"   "PNPT1"   
 [67] "PROCR"    "PSMA3"    "PSMB8"    "PSMB9"    "PSME1"    "PSME2"   
 [73] "RIPK2"    "RNF31"    "RSAD2"    "RTP4"     "SAMD9"    "SAMD9L"  
 [79] "SELL"     "SLC25A28" "SP110"    "STAT2"    "TAP1"     "TDRD7"   
 [85] "TENT5A"   "TMEM140"  "TRAFD1"   "TRIM14"   "TRIM21"   "TRIM25"  
 [91] "TRIM26"   "TRIM5"    "TXNIP"    "UBA7"     "UBE2L6"   "USP18"   
 [97] "WARS1"    "APOL6"    "ARID5B"   "ARL4A"    "AUTS2"    "BANK1"   
[103] "BPGM"     "BTG1"     "C1R"      "CASP3"    "CASP4"    "CASP7"   
[109] "CCL2"     "CCL5"     "CCL7"     "CD274"    "CD38"     "CD40"    
[115] "CD69"     "CD86"     "CDKN1A"   "CFB"      "CFH"      "CIITA"   
[121] "CMKLR1"   "CSF2RB"   "CXCL9"    "EIF4E3"   "FAS"      "FCGR1A"  
[127] "FGL2"     "FPR1"     "GBP6"     "GCH1"     "GPR18"    "GZMA"    
[133] "HIF1A"    "HLA-A"    "HLA-B"    "HLA-DMA"  "HLA-DQA1" "HLA-DRB1"
[139] "HLA-G"    "ICAM1"    "IDO1"     "IFIT1"    "IFNAR2"   "IL10RA"  
[145] "IL15RA"   "IL18BP"   "IL2RB"    "IL6"      "IRF4"     "IRF5"    
[151] "IRF8"     "ISOC1"    "ITGB7"    "JAK2"     "KLRK1"    "LATS2"   
[157] "LCP2"     "LYSMD2"   "MARCHF1"  "METTL7B"  "MT2A"     "MTHFD2"  
[163] "MVP"      "MX2"      "MYD88"    "NAMPT"    "NCOA3"    "NFKB1"   
[169] "NFKBIA"   "NLRC5"    "NOD1"     "NUP93"    "OAS2"     "OAS3"    
[175] "P2RY14"   "PDE4B"    "PELI1"    "PFKP"     "PIM1"     "PLA2G4A" 
[181] "PML"      "PNP"      "PSMA2"    "PSMB10"   "PSMB2"    "PTGS2"   
[187] "PTPN1"    "PTPN2"    "PTPN6"    "RAPGEF6"  "RBCK1"    "RIGI"    
[193] "RIPK1"    "RNF213"   "SAMHD1"   "SECTM1"   "SELP"     "SERPING1"
[199] "SLAMF7"   "SOCS1"    "SOCS3"    "SOD2"     "SPPL2A"   "SRI"     
[205] "SSPN"     "ST3GAL5"  "ST8SIA4"  "STAT1"    "STAT3"    "STAT4"   
[211] "TAPBP"    "TNFAIP2"  "TNFAIP3"  "TNFAIP6"  "TNFSF10"  "TOR1B"   
[217] "UPP1"     "VAMP5"    "VAMP8"    "VCAM1"    "XAF1"     "XCL1"    
[223] "ZBP1"     "ZNFX1"    "HSD17B1"  "CA2"      "CCNA1"    "CXCL1"   
[229] "GALC"     "IFI6"     "MCL1"     "ROBO1"    "SLC6A15"  "THBS1"   
[235] "TIMP3"    "DDX58"    "WARS"     "MARCH1"   "PRIC285"  "FTSJD2"  
[241] "NOS2"     "PDCD1LG2" "LGALS9"   "HMGB1"    "CEACAM1"  "FGL1"    
[247] "HLA-DRA"  "NECTIN2"  "PVR"      "SIGLEC10" "TNFRSF14" "VSIG4"   
[253] "VSIR"     "HLA-E"    "LGALS3"   "CD80"     "LILRB1"   "IL10"    
[259] "CCL22"   

$Minn_Benci_ISG.RS
 [1] "IFI27"    "IRF7"     "USP18"    "BST2"     "CXCL10"   "DDX60"   
 [7] "HERC6"    "HLA-B"    "HLA-G"    "IFI35"    "IFI44"    "IFI44L"  
[13] "IFIT1"    "IFIT3"    "ISG15"    "LGALS3BP" "LY6E"     "MX1"     
[19] "MX2"      "OAS3"     "OASL"     "PLSCR1"   "STAT1"    "TRIM14"  
[25] "HSD17B1"  "OAS1"     "CA2"      "CCNA1"    "CXCL1"    "GALC"    
[31] "IFI6"     "IFITM1"   "LAMP3"    "MCL1"     "ROBO1"    "SLC6A15" 
[37] "THBS1"    "TIMP3"   

$Minn_Benci_IFNG.GS
  [1] "TNFSF10"  "IRF9"     "PSMB9"    "EPSTI1"   "PARP12"   "TRIM25"  
  [7] "LAP3"     "CASP7"    "UPP1"     "B2M"      "IRF4"     "SRI"     
 [13] "NFKBIA"   "IFIT2"    "OAS2"     "TAP1"     "EIF2AK2"  "RSAD2"   
 [19] "IRF1"     "XAF1"     "SP110"    "PSMB8"    "IFITM3"   "GBP4"    
 [25] "IRF8"     "PML"      "IFIH1"    "UBE2L6"   "ADAR"     "STAT2"   
 [31] "CXCL9"    "

In [66]:
writeGMT <- function #Create a gmt (gene matrix transposed) file
### Createss a gmt (gene matrix transposed) file such as those
### provided by mSigDB or geneSigDB, from an R list object.
### Function by Levi Waldron.
(object,
### R list object that will be converted to GMT file.  Each element
### should contain a vector of gene names, and the names of the
### elements will used for the gene set names
 fname
### Output file name for .gmt file
 ){
  if (class(object) != "list") stop("object should be of class 'list'")
  if(file.exists(fname)) unlink(fname)
  for (iElement in 1:length(object)){
    write.table(t(c(make.names(rep(names(object)[iElement],2)),object[[iElement]])),
                sep="\t",quote=FALSE,
                file=fname,append=TRUE,col.names=FALSE,row.names=FALSE)
  }
### Called for the effect of writing a .gmt file
}
writeGMT(supp_table2_sigs, fname = "../../Tables/Supp_Table2_Signatures.txt")

### Supp Table 3. Significant mixed effects IFN-inducibility model results for all cell types in full dataset (tx and no-tx) <a name="3"></a>

In [72]:
celltypes = unique(as.character(seur_full$BBK_Broad_Celltype_DE))
celltypes = setdiff(celltypes, c("Omitted","Contam"))
celltypes

[1] "Tumor"                       "Monocyte"                   
 [3] "TAM"                         "CD8"                        
 [5] "Plasma cell"                 "CD8 NK-like"                
 [7] "DC"                          "Endothelial"                
 [9] "T-reg"                       "NK"                         
[11] "T-Helper"                    "B cell"                     
[13] "Perivascular_and_Fibroblast"

In [80]:
for (i in 1:length(celltypes)){
    me_result = readRDS(paste("../../Outputs/Manuscript_MixedEffects_Output_Tx_NoTx_Stage_Corrected_EarlyLate/",celltypes[i],"_CytoSig_MixedEffects_Tx_NoTx_Patient_Stage_Corrected_EarlyLate_Vargenes.RDS", sep = ""))
    me_result = me_result[,c("IFN1_resp_beta","IFNG_resp_beta","IFN1_resp_pval","IFNG_resp_pval","IFN1_resp_p_adj_BH","IFNG_resp_p_adj_BH","gene")]
    me_result = subset(me_result, IFN1_resp_beta > 0 & IFN1_resp_p_adj_BH < 0.01 | IFNG_resp_beta > 0 & IFNG_resp_p_adj_BH < 0.01)
    
    write.table(me_result, file = paste("../../Tables/Supp_Table3_Tabs/Supp_Table3_",celltypes[i],".txt", sep = ""),sep = '\t',quote = F, col.names = NA)
}

### Supp Table 4. DEG between CD8 subsets <a name="4"></a>

In [5]:
markers_cd8 = readRDS(file = "../../Supp_Figures/Supp_Figure3_4/CD8_DE.RDS")
head(markers_cd8)

,p_val,avg_log2FC,pct.1,pct.2,p_val_adj,cluster,gene
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<chr>
CCL5,2.090318e-37,0.4335790,0.966,0.961,8.281630e-33,CD8_RPS,CCL5
RPS3,5.912126e-32,0.2892803,0.989,0.995,2.342325e-27,CD8_RPS,RPS3
RPL41,4.639500e-31,0.3225406,0.994,0.998,1.838124e-26,CD8_RPS,RPL41
RPS27,3.714119e-29,0.4552496,0.977,0.992,1.471497e-24,CD8_RPS,RPS27
RPL30,5.266161e-28,0.3190091,0.976,0.993,2.086400e-23,CD8_RPS,RPL30
RPL28,1.245668e-26,0.2730389,0.989,0.998,4.935211e-22,CD8_RPS,RPL28


In [6]:
table(markers_cd8$cluster)


       CD8_RPS       CD8_IL7R      CD8_KLRB1   CD8_HSP_IFNG     CD8_ISG_Hi 
            30            195            250            115            122 
      CD8_LAG3 CD8_TNFRSF9_Hi      CD8_FOXP3    CD8_TRBV7-7    CD8_TRBV5-6 
           123            299            113             37             58 
   CD8_TRBV3-1  CD8_Cycling_1  CD8_Cycling_2  CD8_Cycling_4 
           109            713            818            705 

In [7]:
write.table(markers_cd8, file = "../../Tables/Supp_Table4_CD8s_DE.txt", sep = '\t',quote = F, col.names = NA)

### Supp Table 5. DEG between non-cycling TAM subsets <a name="5"></a>

In [8]:
markers_tam = readRDS(file = "../../Supp_Figures/Supp_Figure5_6//TAM_DE.RDS")
head(markers_tam)

,p_val,avg_log2FC,pct.1,pct.2,p_val_adj,cluster,gene
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<chr>
CXCL10,0,3.776016,0.623,0.081,0,TAM_CXCL10,CXCL10
CXCL9,0,2.594469,0.517,0.116,0,TAM_CXCL10,CXCL9
ISG15,0,2.285942,0.712,0.374,0,TAM_CXCL10,ISG15
GBP1,0,1.986370,0.849,0.329,0,TAM_CXCL10,GBP1
IFITM1,0,1.791747,0.383,0.083,0,TAM_CXCL10,IFITM1
CXCL11,0,1.616437,0.305,0.022,0,TAM_CXCL10,CXCL11


In [9]:
table(markers_tam$cluster)


 TAM_CXCL10      TAM_C3    TAM_CCL4 TAM_SELENOP    TAM_SPP1     TAM_HSP 
        235         126         240         155         243         328 

In [10]:
write.table(markers_tam, file = "../../Tables/Supp_Table5_TAMs_DE.txt", sep = '\t',quote = F, col.names = NA)

### Supp Table 6. Significant mixed effects IFN-inducibility model results for all cell types in untreated subset of dataset <a name="6"></a>

In [2]:
seur_full = readRDS(file = "../../Data_Objects/seur_full_BBK_Manuscript.RDS")

In [3]:
celltypes = unique(as.character(seur_full$BBK_Broad_Celltype_DE))
celltypes = setdiff(celltypes, c("Omitted","Contam"))
celltypes

[1] "Tumor"                       "Monocyte"                   
 [3] "TAM"                         "CD8"                        
 [5] "Plasma cell"                 "CD8 NK-like"                
 [7] "DC"                          "Endothelial"                
 [9] "T-reg"                       "NK"                         
[11] "T-Helper"                    "B cell"                     
[13] "Perivascular_and_Fibroblast"

In [5]:
for (i in 1:length(celltypes)){
    me_result = readRDS(paste("../../Outputs/Manuscript_MixedEffects_Output_Stage_Corrected_EarlyLate/",celltypes[i],"_CytoSig_MixedEffects_Patient_Stage_Corrected_EarlyLate_Vargenes.RDS", sep = ""))
    me_result = me_result[,c("IFN1_resp_beta","IFNG_resp_beta","IFN1_resp_pval","IFNG_resp_pval","IFN1_resp_p_adj_BH","IFNG_resp_p_adj_BH","gene")]
    me_result = subset(me_result, IFN1_resp_beta > 0 & IFN1_resp_p_adj_BH < 0.01 | IFNG_resp_beta > 0 & IFNG_resp_p_adj_BH < 0.01)
    
    write.table(me_result, file = paste("../../Tables/Supp_Table6_Tabs/Supp_Table6_",celltypes[i],".txt", sep = ""),sep = '\t',quote = F, col.names = NA)
}